In [1]:
# ---- PRE-CHECK: Purge cursed variables ----
try:
    del pandas
except:
    pass


In [2]:
# ---- Imports ----
import pandas as pd

In [3]:
# ---- Load Raw File ----
df = pd.read_csv('raw_data.csv')  # Replace with your actual filename

In [4]:
# ---- Drop Empty Rows/Columns ----
df.dropna(axis=0, how='all', inplace=True)
df.dropna(axis=1, how='all', inplace=True)

In [5]:
# ---- Clean Column Names ----
df.columns = df.columns.str.strip()
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [6]:
# ---- Convert wt% to ppm for Known Elements ----
for element in ['Ca', 'K', 'Fe']:
    if element in df.columns and df[element].max() < 1:
        df[element + '_ppm'] = df[element] * 10000

In [7]:
# ---- Handle Divide-by-Zero Risk in Denominator Columns ----
for col in ['Al', 'Ti']:
    if col in df.columns:
        df[col] = df[col].replace(0, 0.0001)

In [8]:
# ---- Clip Outliers (1st - 99th Percentile) ----
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    q_low = df[col].quantile(0.01)
    q_hi = df[col].quantile(0.99)
    df[col] = df[col].clip(lower=q_low, upper=q_hi)

In [9]:
# ---- Save Processed File ----
df.to_csv('processed_data.csv', index=False)
print("✅ Cleaned file saved as 'processed_data.csv'")

✅ Cleaned file saved as 'processed_data.csv'
